# Weights & Biases optimalizácia

Súbor: wandb_optimalizacia.ipynb

Program: Hospodárska informatika

Vypracovala:  Veronika Motúzová 

Bakalárska práca: Metódy hlbokého učenia pre detekciu meteorov v astronomických snímkach

Vedúci bakalárskej práce: doc. Ing. Peter Butka, PhD.

Konzultant: Ing. Viera Maslej Krešňáková

výsledky: https://wandb.ai/motuziik/model_optimization

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.layers import Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, AveragePooling2D, GlobalAveragePooling2D,ZeroPadding2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import array
from sklearn.metrics import confusion_matrix, classification_report
import itertools 
from sklearn import metrics
from tensorflow.keras.callbacks import ModelCheckpoint 
from sklearn.model_selection import train_test_split
from wandb.keras import WandbCallback
import wandb

In [3]:
!wandb login

wandb: Currently logged in as: motuziik (use `wandb login --relogin` to force relogin)


Načítanie a rozdelenie dát

In [4]:
train = np.load('../data/data_final_train.npz')
test = np.load('../data/data_final_test.npz')
x_train = train['train_data'].reshape(train['train_data'].shape[0], 128, 128, 1)
x_test = test['test_data'].reshape(test['test_data'].shape[0], 128, 128, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = to_categorical(train['train_targets'], 2)
y_test = to_categorical(test['test_targets'], 2)
X = np.concatenate([x_train,x_test])
Y = np.concatenate([y_train,y_test])
x_tr, x_test, y_tr, y_test = train_test_split(X,Y,test_size=0.2)
x_train, x_val, y_train, y_val = train_test_split(x_tr, y_tr, test_size=0.2)

In [5]:
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)
print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(30394, 128, 128, 1)
(9499, 128, 128, 1)
(7599, 128, 128, 1)
(30394, 2)
(9499, 2)
(7599, 2)


#### Sweep file

In [6]:
sweep_config = {
    "name": "model_1",
    "method": "grid",
    "metric": {"name": "val_acc", "goal": "maximize"}
}

parameters_dict = {
    'optimizer': {'values': ['adam','SGD','RMSprop']},
    'batch_size': {'values': [128, 64]},
    'dropout': {'values': [0.25, 0.2, 0.15]},
    'activation': {'values': ['relu', 'tanh']},
    }
    
sweep_config['parameters'] = parameters_dict

sweep_id = wandb.sweep(sweep_config, entity = "motuziik", project = "model_optimization")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 83eceyoz
Sweep URL: https://wandb.ai/motuziik/model_optimization/sweeps/83eceyoz


In [7]:
sweep_config['parameters']

{'optimizer': {'values': ['adam', 'SGD', 'RMSprop']},
 'batch_size': {'values': [128, 64]},
 'dropout': {'values': [0.25, 0.2, 0.15]},
 'activation': {'values': ['relu', 'tanh']}}

In [8]:
wandb.sweep

<function wandb.wandb_controller.sweep(sweep, entity=None, project=None)>

In [9]:
def train():

    config_defaults = {
        'batch_size': 128,
        'optimizer': 'adam',
        'dropout': 0.25,
        'activation': 'relu',
     }

    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    
    config = wandb.config
    
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(128, 128,1)))
    model.add(Conv2D(64, 3, strides=1, activation=config.activation)) 
    model.add(MaxPool2D(pool_size = 3, strides = 2))
    model.add(Conv2D(64, 5, strides=1, padding='same', activation=config.activation))
    model.add(MaxPool2D(pool_size = 3, strides = 2))
    model.add(Conv2D(64, 3, strides=1, padding='same', activation=config.activation)) 
    model.add(Conv2D(64, 3, strides=1, padding='same', activation=config.activation))
    model.add(MaxPool2D(pool_size = 3, strides = 2))
    model.add(Flatten())
    model.add(Dense(96, activation=config.activation))
    model.add(Dropout(config.dropout)) 
    model.add(Dense(96, activation=config.activation))
    model.add(Dropout(config.dropout)) 
    model.add(Dense(2, activation='softmax'))
    
    model.compile(loss = 'categorical_crossentropy', optimizer = config.optimizer, metrics = ['accuracy'])
    
    model.fit(x_train, y_train, validation_data=(x_val, y_val) , epochs = 15, batch_size = config.batch_size, callbacks=[WandbCallback(validation_data=(x_val, y_val))]) 

### sweep

In [10]:
# Initialize a new sweep
# Arguments:
#     – sweep_id: the sweep_id to run - this was returned above by wandb.sweep()
#     – function: function that defines your model architecture and trains it
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: fuwqm1xu with config:
wandb: 	activation: relu
wandb: 	batch_size: 128
wandb: 	dropout: 0.25
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: motuziik (use `wandb login --relogin` to force relogin)


Epoch 1/15
238/238 [==============================] - 53s 221ms/step - loss: 0.3858 - accuracy: 0.8217 - val_loss: 0.2466 - val_accuracy: 0.8988
Epoch 2/15
238/238 [==============================] - 51s 215ms/step - loss: 0.2327 - accuracy: 0.9091 - val_loss: 0.2041 - val_accuracy: 0.9228
Epoch 3/15
238/238 [==============================] - 51s 216ms/step - loss: 0.1752 - accuracy: 0.9395 - val_loss: 0.1393 - val_accuracy: 0.9507
Epoch 4/15
238/238 [==============================] - 51s 214ms/step - loss: 0.1556 - accuracy: 0.9460 - val_loss: 0.1778 - val_accuracy: 0.9346
Epoch 5/15
238/238 [==============================] - 51s 215ms/step - loss: 0.1247 - accuracy: 0.9597 - val_loss: 0.0947 - val_accuracy: 0.9687
Epoch 6/15
238/238 [==============================] - 51s 215ms/step - loss: 0.1008 - accuracy: 0.9674 - val_loss: 0.0944 - val_accuracy: 0.9705
Epoch 7/15
238/238 [==============================] - 51s 215ms/step - loss: 0.0861 - accuracy: 0.9730 - val_loss: 0.0807 - val_ac

epoch,14
loss,0.0374
accuracy,0.98809
val_loss,0.12573
val_accuracy,0.97118
_runtime,781
_timestamp,1619435141
_step,14
best_val_loss,0.07154
best_epoch,8


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▅▄▃▃▂▂▂▂▂▁▁▁▁▁
accuracy,▁▅▆▆▇▇▇████████
val_loss,█▆▄▅▂▂▁▂▁▂▂▂▃▂▃
val_accuracy,▁▃▆▄▇▇█▇█▇▇█▇█▇
_runtime,▁▁▂▃▃▄▄▅▅▆▆▇▇██
_timestamp,▁▁▂▃▃▄▄▅▅▆▆▇▇██
_step,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█


wandb: Agent Starting Run: c2xw2yio with config:
wandb: 	activation: relu
wandb: 	batch_size: 128
wandb: 	dropout: 0.25
wandb: 	optimizer: SGD
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

wandb: Agent Starting Run: 101pavhn with config:
wandb: 	activation: tanh
wandb: 	batch_size: 64
wandb: 	dropout: 0.15
wandb: 	optimizer: RMSprop
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/15
475/475 [==============================] - 57s 119ms/step - loss: 0.4578 - accuracy: 0.7908 - val_loss: 0.3684 - val_accuracy: 0.8418
Epoch 2/15
475/475 [==============================] - 56s 118ms/step - loss: 0.3387 - accuracy: 0.8523 - val_loss: 0.3198 - val_accuracy: 0.8621
Epoch 3/15
475/475 [==============================] - 56s 118ms/step - loss: 0.2815 - accuracy: 0.8819 - val_loss: 0.2174 - val_accuracy: 0.9159
Epoch 4/15
475/475 [==============================] - 56s 118ms/step - loss: 0.2294 - accuracy: 0.9114 - val_loss: 0.2004 - val_accuracy: 0.9304
Epoch 5/15
475/475 [==============================] - 56s 118ms/step - loss: 0.2002 - accuracy: 0.9231 - val_loss: 0.2363 - val_accuracy: 0.9229
Epoch 6/15
475/475 [==============================] - 56s 118ms/step - loss: 0.1753 - accuracy: 0.9354 - val_loss: 0.1362 - val_accuracy: 0.9496
Epoch 7/15
475/475 [==============================] - 56s 118ms/step - loss: 0.1677 - accuracy: 0.9398 - val_loss: 0.1413 - val_ac

epoch,14
loss,0.12133
accuracy,0.9591
val_loss,0.11202
val_accuracy,0.96171
_runtime,849
_timestamp,1619463615
_step,14
best_val_loss,0.11202
best_epoch,14


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▆▄▃▃▂▂▂▂▁▁▁▁▁▁
accuracy,▁▄▅▆▇▇▇▇▇██████
val_loss,█▇▄▃▄▂▂▂▁▂▁▂▂▂▁
val_accuracy,▁▂▅▆▆▇▇▇█▇█▇█▇█
_runtime,▁▁▂▃▃▃▄▅▅▆▆▆▇▇█
_timestamp,▁▁▂▃▃▃▄▅▅▆▆▆▇▇█
_step,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [1]:
 !nvidia-smi

Mon Apr 26 10:51:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K40c          Off  | 00000000:01:00.0 Off |                    0 |
| 28%   57C    P0    72W / 235W |      0MiB / 11441MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------